In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np
import time
import random
from itertools import combinations
from tqdm.notebook import tqdm
import optuna
import math

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer
from sklearn.compose import make_column_transformer
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.feature_selection import RFECV, mutual_info_classif

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from concurrent.futures import ThreadPoolExecutor
from category_encoders import TargetEncoder

from lightgbm import LGBMClassifier

from pprint import pprint
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiment_name = 'lgbm'

In [2]:
train = pd.read_csv('new_combined_train_spear.csv')

train.head()

,FAF_ceil_round,FCVC_ceil_round,TUE_floor_round,TUE_ceil_round,CH2O_floor_round,FAF_floor_round,CH2O_ceil_round,FCVC_round,NCP_ceil_round,FAF_round,FCVC_floor_round,TUE_round,CH2O_round,NCP_floor_round,Weight_2dec,Height_2dec,Automobile,Bike,Motorbike,Walking,CALC_map,family_history_with_overweight_map,CAEC_map,SCC_map,Gender_map,FAVC_map,SMOKE_map,CALC,CAEC,Weight_2dec_gt_Height_2dec,Weight_2dec_gt_Gender,Weight_2dec_div_CALC,Weight_2dec_gt_CALC,Weight_2dec_div_CAEC,Weight_2dec_gt_CAEC,Weight_2dec_gt_MTRANS,Weight_2dec_div_family_history_with_overweight,Weight_2dec_gt_family_history_with_overweight,Weight_2dec_div_FAVC,Weight_2dec_gt_FAVC,Weight_2dec_gt_SMOKE,Weight_2dec_div_SCC,Weight_2dec_gt_SCC,Height_2dec_x_Gender,Height_2dec_plus_Gender,Height_2dec_gt_Gender,Height_2dec_x_CALC,Height_2dec_minus_CALC,Height_2dec_gt_CALC,Height_2dec_x_CAEC,Height_2dec_minus_CAEC,Height_2dec_x_MTRANS,Height_2dec_minus_MTRANS,Height_2dec_x_family_history_with_overweight,Height_2dec_minus_family_history_with_overweight,Height_2dec_x_FAVC,Height_2dec_minus_FAVC,Height_2dec_gt_FAVC,Height_2dec_gt_SMOKE,Gender_x_CALC,Gender_minus_CALC,Gender_x_CAEC,Gender_minus_CAEC,Gender_minus_MTRANS,Std_Gender_MTRANS,Gender_x_family_history_with_overweight,Gender_minus_family_history_with_overweight,Gender_x_FAVC,Gender_minus_FAVC,Gender_x_SCC,CALC_x_CAEC,CALC_minus_CAEC,CALC_x_MTRANS,CALC_minus_MTRANS,CALC_x_family_history_with_overweight,CALC_minus_family_history_with_overweight,CALC_x_FAVC,CALC_minus_FAVC,CAEC_x_MTRANS,CAEC_minus_MTRANS,CAEC_x_family_history_with_overweight,CAEC_minus_family_history_with_overweight,CAEC_div_family_history_with_overweight,Std_CAEC_family_history_with_overweight,CAEC_gt_family_history_with_overweight,CAEC_x_FAVC,CAEC_minus_FAVC,MTRANS_x_family_history_with_overweight,MTRANS_minus_family_history_with_overweight,MTRANS_x_FAVC,MTRANS_minus_FAVC,Std_MTRANS_SMOKE,MTRANS_x_SCC,MTRANS_minus_SCC,Std_family_history_with_overweight_SCC,FAVC_x_SCC,FAVC_minus_SCC,SMOKE_x_SCC,SMOKE_minus_SCC,Weight_2dec_div_prod_of_CALC_and_CAEC,Weight_2dec_div_prod_of_CALC_and_family_history_with_overweight,Weight_2dec_div_prod_of_CALC_and_FAVC,Weight_2dec_div_prod_of_CALC_and_SCC,Weight_2dec_div_sum_of_CAEC_and_family_history_with_overweight,Weight_2dec_div_prod_of_CAEC_and_family_history_with_overweight,CAEC_div_prod_of_Weight_2dec_and_family_history_with_overweight,family_history_with_overweight_div_prod_of_Weight_2dec_and_CAEC,Weight_2dec_div_prod_of_CAEC_and_FAVC,Height_2dec_x_Gender_x_CALC,CALC_div_prod_of_Height_2dec_and_Gender,Height_2dec_plus_Gender_minus_CAEC,Height_2dec_plus_Gender_plus_CAEC,Height_2dec_x_Gender_x_CAEC,CAEC_div_prod_of_Height_2dec_and_Gender,Std_of_Height_2dec_Gender_CAEC,Height_2dec_plus_Gender_minus_MTRANS,Height_2dec_plus_Gender_plus_MTRANS,Height_2dec_x_Gender_x_MTRANS,MTRANS_div_prod_of_Height_2dec_and_Gender,Height_2dec_plus_Gender_minus_family_history_with_overweight,Height_2dec_plus_Gender_plus_family_history_with_overweight,Height_2dec_x_Gender_x_family_history_with_overweight,family_history_with_overweight_div_prod_of_Height_2dec_and_Gender,Height_2dec_plus_Gender_minus_FAVC,Height_2dec_plus_Gender_plus_FAVC,Height_2dec_x_Gender_x_FAVC,FAVC_div_prod_of_Height_2dec_and_Gender,Gender_plus_SCC_minus_Height_2dec,Height_2dec_plus_CALC_minus_CAEC,Height_2dec_plus_CAEC_minus_CALC,CALC_plus_CAEC_minus_Height_2dec,Height_2dec_plus_CALC_plus_CAEC,Height_2dec_plus_MTRANS_minus_CALC,Height_2dec_plus_CALC_minus_family_history_with_overweight,Height_2dec_plus_family_history_with_overweight_minus_CALC,CALC_plus_family_history_with_overweight_minus_Height_2dec,Height_2dec_plus_CALC_plus_family_history_with_overweight,Height_2dec_plus_CALC_minus_FAVC,Height_2dec_plus_FAVC_minus_CALC,CALC_plus_FAVC_minus_Height_2dec,Height_2dec_plus_MTRANS_minus_CAEC,Height_2dec_plus_CAEC_plus_MTRANS,Height_2dec_plus_CAEC_minus_family_history_with_overweight,Height_2dec_plus_family_history_with_overweight_minus_CAEC,CAEC_plus_family_history_with_over

In [3]:
TARGET = 'NObeyesdad'

In [4]:
X = train.drop(TARGET, axis=1)
y = train[TARGET]

sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

In [5]:
# cat_cols_11 = X.columns[X.nunique() < 11].tolist()
# cat_cols_7 = X.columns[X.nunique() < 7].tolist()
# diff = list(set(cat_cols_11) - set(cat_cols_7))

# X['CAEC_minus_FAVC'].unique()

X.apply(lambda col: col.nunique() / X.shape[0] * 100).sort_values(ascending=False)

# cat_cols = X.columns[X.nunique() < 11]
# num_cols = X.columns[X.nunique() >= 11]
# len(cat_cols), len(num_cols)

Weight_2dec_div_prod_of_CALC_and_CAEC                                13.424286
Weight_2dec_div_prod_of_CALC_and_FAVC                                12.847086
Weight_2dec_div_prod_of_CALC_and_family_history_with_overweight      12.733395
Weight_2dec_div_prod_of_CALC_and_SCC                                 12.103721
Weight_2dec_div_CALC                                                 11.194193
Weight_2dec_div_prod_of_CAEC_and_FAVC                                10.481438
family_history_with_overweight_div_prod_of_Weight_2dec_and_CAEC      10.450829
Weight_2dec_div_sum_of_CAEC_and_family_history_with_overweight       10.450829
Weight_2dec_div_prod_of_CAEC_and_family_history_with_overweight      10.450829
CAEC_div_prod_of_Weight_2dec_and_family_history_with_overweight      10.450829
Weight_2dec_div_CAEC                                                  8.789191
Weight_2dec_div_FAVC                                                  8.588045
Weight_2dec_div_family_history_with_overweight      

In [6]:
# transformer = make_column_transformer(
#     (StandardScaler(), num_cols),
#     remainder='passthrough')

# df_to_ohe = X.copy()
# # test_ohe = test.drop('id', axis=1)

# transformed = transformer.fit_transform(df_to_ohe)

# transformed = transformer.transform(X)

# # Get the transformed feature names
# transformed_feat_names = [name.split('__')[-1] for name in transformer.get_feature_names_out()]

# # Create DataFrame of the transformed features
# df_to_ohe_transformed = pd.DataFrame(transformed, columns=transformed_feat_names)
# df_to_ohe_transformed.sample()

- Mutual Information

In [7]:
# Generate a random feature
np.random.seed(5)
X['random_control_feature'] = np.round(np.random.uniform(-2, 2, X.shape[0]), 6)
X.shape

(22869, 347)

In [13]:
# Calculate mutual information (3, 5, 10, 15, 20) - this can be memory and CPU intensive
# The plan is to remove any features that have mutual information less than random_feature because surely you cannot be important
mi_scores = mutual_info_classif(X, y, discrete_features='auto', n_neighbors=5, copy=True, random_state=5)

# Make results easier to interpret by placing them in a DataFrame
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

# Now you have the MI scores sorted from the most to the least informative
mi_scores.to_csv(f'{experiment_name}_5_mi_scores.csv')

In [15]:
# Read the CSV
mi_df = pd.read_csv('lgbm_mi_scores.csv')

# Get the mi score for control feature
mi_threshold = mi_df.loc[mi_df['Feature'] == 'random_control_feature', 'Average'].iloc[0]

# Get the features above random_control_feature to move forward
mi_feats = mi_df[mi_df['Average'] > mi_threshold]['Feature'].tolist()

In [16]:
X_mi = X[mi_feats]

- Feature Importances

In [24]:
n_splits = 10
feat_import_model = LGBMClassifier(n_jobs=-1, random_state=5, objective='multiclass', num_class=y.nunique())

# Initialize array to store feature importances
feature_importances = np.zeros(X_mi.shape[1])

# Loop through each fold and calculate the feature importances
for train_index, test_index in sk.split(X_mi, y):
    X_train, X_test = X_mi.iloc[train_index], X_mi.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    feat_import_model.fit(X_train, y_train)

    # Get the feature importances and them to the total
    feature_importances += feat_import_model.feature_importances_

feature_importances /= n_splits

feature_importances_dict = dict(zip(X_mi.columns, feature_importances))

df = pd.DataFrame.from_dict(feature_importances_dict, orient='index')
df = df.rename(columns={0: 'Avg_Feat_Importance'})
df.sort_values(by='Avg_Feat_Importance', ascending=False, inplace=True)
df.to_csv('feature_importances.csv')

In [25]:
# Read the CSV
fi_df = pd.read_csv('feature_importances.csv')

# Define threshold feature importance score
fi_threshold = 0

# Get the features above threshold
fi_feats = fi_df[fi_df['Avg_Feat_Importance'] > fi_threshold]['Feature'].tolist()

In [26]:
X_fi = X[fi_feats]

In [27]:
models = [LGBMClassifier(n_jobs=-1, random_state=5, objective='multiclass', num_class=y.nunique()),]

- Permutation Importance

In [31]:
# Generate a random feature for X
np.random.seed(5)
X_mi['random_control_feature'] = np.round(np.random.uniform(-2, 2, X_mi.shape[0]), 6)
X_mi.shape

(22869, 339)

In [32]:
perm_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=5)

perm_importances = {model.__class__.__name__: [] for model in models}

for i, (train_idx, test_idx) in enumerate(perm_cv.split(X_mi, y)):
    X_train, X_test = X_mi.iloc[train_idx], X_mi.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        # Calculate permutation importance
        result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=5, n_jobs=-1, scoring='accuracy')
        perm_importances[model_name].append(result.importances_mean)
        print(f'Done with {model_name}.')
    
    print(f'Done with Fold {i+1}', end='\n\n')

Done with LGBMClassifier.
Done with Fold 1

Done with LGBMClassifier.
Done with Fold 2

Done with LGBMClassifier.
Done with Fold 3

Done with LGBMClassifier.
Done with Fold 4

Done with LGBMClassifier.
Done with Fold 5

Done with LGBMClassifier.
Done with Fold 6

Done with LGBMClassifier.
Done with Fold 7

Done with LGBMClassifier.
Done with Fold 8

Done with LGBMClassifier.
Done with Fold 9

Done with LGBMClassifier.
Done with Fold 10



In [35]:
# Average importances across folds and export to CSV
for model_name, importances in perm_importances.items():
    avg_importance = np.mean(importances, axis=0)
    importance_df = pd.DataFrame({'Feature': X_mi.columns, 'Importance': avg_importance})
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    # Export to CSV
    importance_df.to_csv(f'.\permutation_importances\{model_name}_permutation_importance.csv', index=False)

print('Done with Permuation Importances', end='\n\n')

Done with Permuation Importances



In [36]:
directory = 'permutation_importances'

# Initialize a dictionary for the features
perm_important_features = {}

for model in models:
    model_name = model.__class__.__name__
    csv_path = os.path.join(directory, f'{model_name}_permutation_importance.csv')
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)

        # Check for 'random_control_feature' and its importance
        if 'random_control_feature' in df['Feature'].values:
            random_feature_importance = df.loc[df['Feature'] == 'random_control_feature', 'Importance'].iloc[0]
        else:
            random_feature_importance = 0

        # Determine the threshold
        threshold = max(0, random_feature_importance)

        # Filter features where importance is greater than 0
        important_feats = df[df['Importance'] > threshold]['Feature'].tolist()

        # important_feats = df.head(50)['Feature'].tolist()

        # Add to importance dictionary
        perm_important_features[model_name] = important_feats

    else:
        print(f'CSV file for {model_name} not found.')

print('Done getting important features dictionary')

Done getting important features dictionary


- RFECV

In [40]:
rfe_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Initialize empty dictionary for RFECV features
rfecv_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__
		
    features = perm_important_features[MLA_name]

    # incase there is no feature that had importance, go to the next model
    if len(features) == 0:
        continue
	
    X_rfecv = X[features]

    try:
        print(f'Starting with {MLA_name}')
        # Create the RFECV object and rank each feature
        selector = RFECV(alg, cv=sk, step=1, scoring='accuracy', verbose=2)
        selector = selector.fit(X_rfecv, y)

        selected_features = list(X_rfecv.columns[selector.support_])

        rfecv_features[MLA_name] = selected_features

        print(f'Done with {MLA_name}', end='\n\n')
    
    except ValueError:
        rfecv_features[MLA_name] = features
        print(f'{MLA_name} does not have coef_ or feature_importances_', end='\n\n')


Starting with LGBMClassifier
Fitting estimator with 178 features.
Fitting estimator with 177 features.
Fitting estimator with 176 features.
Fitting estimator with 175 features.
Fitting estimator with 174 features.
Fitting estimator with 173 features.
Fitting estimator with 172 features.
Fitting estimator with 171 features.
Fitting estimator with 170 features.
Fitting estimator with 169 features.
Fitting estimator with 168 features.
Fitting estimator with 167 features.
Fitting estimator with 166 features.
Fitting estimator with 165 features.
Fitting estimator with 164 features.
Fitting estimator with 163 features.
Fitting estimator with 162 features.
Fitting estimator with 161 features.
Fitting estimator with 160 features.
Fitting estimator with 159 features.
Fitting estimator with 158 features.
Fitting estimator with 157 features.
Fitting estimator with 156 features.
Fitting estimator with 155 features.
Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting e

In [41]:
with open('rfecv_features_lgbm.txt', mode='w') as f:
    pprint(rfecv_features, stream=f)

- SFs

In [43]:
# Initialize empty dictionary for SFS features
sfs_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__

    try:
            
        # features = rfecv_features[MLA_name]
        features = rfecv_features[MLA_name]

        # incase there is no feature that had importance, go to the next model
        if len(features) == 0:
            continue
        
        X_sfs = X[features]

        print(f'Running backward feature selection with {MLA_name}')

        sfs = SFS(alg,
            k_features='best',
            forward=False,
            floating=False,
            scoring='accuracy',
            verbose=2,
            n_jobs=-1,
            cv=sk)
        
        sfs = sfs.fit(X_sfs, y)

        # Get the selected features index
        selected_sfs_idx = list(sfs.k_feature_idx_)

        # Get the feature names
        selected_sfs_feats = X_sfs.columns[selected_sfs_idx]

        sfs_features[MLA_name] = list(selected_sfs_feats)

        print(f'Done with {MLA_name}', end='\n\n')

    except KeyError:
        print(f'{MLA_name} not in the dictionary.')

Running backward feature selection with LGBMClassifier


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  78 out of  78 | elapsed: 18.1min finished

[2024-02-10 21:35:11] Features: 77/1 -- score: 0.9085660295200505[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  77 out of  77 | elapsed: 17.7min finished

[2024-02-10 21:52:56] Features: 76/1 -- score: 0.908085040509345[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  76 out of  76 | elapsed: 17.5min finished

[2024-02-10 22:10:28] Features: 75/1 -- score: 0.9080413342281759[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  75 out of

Done with LGBMClassifier



[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.3s finished

[2024-02-11 05:45:38] Features: 1/1 -- score: 0.7483492249473656

In [44]:
with open('sfs_features.txt', mode='w') as f:
    pprint(sfs_features, stream=f)

In [28]:
def evaluate_models(models, X, y, important_features, cv_split, experiment_name):
    # Create a DataFrame to store comparison results
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train Accuracy', 
                                        'MLA Test Accuracy', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        MLA_name = alg.__class__.__name__
        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train Accuracy': 0,
                'MLA Test Accuracy': 0,
                'MLA Time': "0 min 0.00 sec",
            }

        # Perform cross-validation
        roc_scorer = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False)
        
        cv_results = cross_validate(alg, X[features], y, 
                                    cv=cv_split, 
                                    scoring={'Accuracy': roc_scorer}, 
                                    return_train_score=True, 
                                    n_jobs=-1)

        # Format time
        mean_fit_time = cv_results['fit_time'].mean()
        minutes, seconds = divmod(mean_fit_time, 60)

        print(f'Done with {MLA_name}.')
        
        # Populate results
        return {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train Accuracy': cv_results['train_Accuracy'].mean() if 'train_Accuracy' in cv_results else 0,
            'MLA Test Accuracy': cv_results['test_Accuracy'].mean() if 'test_Accuracy' in cv_results else 0,
            'MLA Time': f"{int(minutes)} min {seconds:.2f} sec",
        }

    results_list = []

    # Use ThreadPoolExecutor for parallel execution
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            if result:
                results_list.append(result)

    # Create a DataFrame from the list of dictionaries
    MLA_compare = pd.DataFrame(results_list)

    # Sort and save results
    MLA_compare.sort_values(by=['MLA Test Accuracy'], ascending=True, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [29]:
baseline_features = {}

for model in models:
    model_name = model.__class__.__name__

    baseline_features[model_name] = list(X_fi.columns)

In [30]:
baseline_models = evaluate_models(models, X_fi, y, baseline_features, sk, f'{experiment_name}')
baseline_models

Done with LGBMClassifier.


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Test Accuracy,MLA Time
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.990325,0.905286,0 min 58.25 sec


In [38]:
# Set seeds for reproducibility
# 22 minutes run time
np.random.seed(42)
random.seed(42)

permutation_models = evaluate_models(models, X, y, perm_important_features, sk, f'{experiment_name}_perm_imp')
permutation_models

Done with LGBMClassifier.


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Test Accuracy,MLA Time
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.98885,0.906467,0 min 30.29 sec


In [42]:
# Set seeds for reproducibility
#  minutes run time
np.random.seed(42)
random.seed(42)

rfecv_models = evaluate_models(models, X, y, rfecv_features, sk, f'{experiment_name}_rfecv')
rfecv_models

Done with LGBMClassifier.


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Test Accuracy,MLA Time
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.98827,0.906467,0 min 15.68 sec


In [45]:
# Set seeds for reproducibility
#  minutes run time
np.random.seed(42)
random.seed(42)

sfs_models = evaluate_models(models, X, y, sfs_features, sk, f'{experiment_name}_sfs')
sfs_models

Done with LGBMClassifier.


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Test Accuracy,MLA Time
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.981984,0.910446,0 min 5.50 sec
